In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

In [16]:
# 1. Đọc dữ liệu
df = pd.read_csv("encoded_data.csv")

# Tách các feature và target
X = df.drop(columns=['gross','gross_log'])
y = df['gross_log']

# Chuẩn hóa dữ liệu (StandardScaler)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Chia dữ liệu thành train và test để kiểm tra overfitting
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print(f"Min gross: {y.min()}")
print(f"Max gross: {y.max()}")
print(f"Mean gross: {y.mean()}")


Min gross: 2.995732273553991
Max gross: 21.79611943720888
Mean gross: 16.81006966826664


In [37]:
#Tạo mô hìnhhình
model = xgb.XGBRegressor(
    objective='reg:squarederror',  # mục tiêu hồi quy
    n_estimators=100,  # Số cây
    max_depth=3 ,       # Chiều sâu tối đa của cây
    learning_rate=0.05, # Tốc độ học
    subsample=0.8,     # Tỷ lệ mẫu ngẫu nhiên
    colsample_bytree=0.8,  # Tỷ lệ cột ngẫu nhiên cho mỗi cây
    random_state=42
)

# Huấn luyện mô hình
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
# Định nghĩa tham số cần tìm kiếm
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# Khởi tạo mô hình
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Khởi tạo GridSearchCV với đánh giá RMSE
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)

# Tiến hành tìm kiếm
grid_search.fit(X_train, y_train)

# In ra tham số tốt nhất
print(f"Best Parameters: {grid_search.best_params_}")

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.8}
Test RMSE: 2.0204
Test R²: 0.5729


In [42]:
# Đánh giá mô hình với tham số tốt nhất
best_model = grid_search.best_estimator_
y_pred_test = best_model.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
r2_test = r2_score(y_test, y_pred_test)

print(f"Test RMSE: {rmse_test:.4f}")
print(f"Test R²: {r2_test:.4f}")

Test RMSE: 2.0204
Test R²: 0.5729


In [38]:
# Dự đoán trên tập huấn luyện và kiểm thử
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

# Đánh giá mô hình với RMSE và R²
train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))

train_r2 = r2_score(y_train, train_pred)
test_r2 = r2_score(y_test, test_pred)

print(f"Training RMSE: {train_rmse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")
print(f"Training R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")

Training RMSE: 1.7367
Test RMSE: 2.0422
Training R²: 0.6803
Test R²: 0.5637


In [41]:
# Cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Đánh giá mô hình bằng cross-validation cho RMSE
cross_val_score_model_rmse = cross_val_score(best_model, X_scaled, y, cv=kfold, scoring='neg_mean_squared_error')
cross_val_rmse = np.sqrt(-cross_val_score_model_rmse.mean())
print(f"Cross-validation RMSE: {cross_val_rmse:.4f}")

# Cross-validation (R²)
cross_val_score_model_r2 = cross_val_score(best_model, X_scaled, y, cv=kfold, scoring='r2')
cross_val_r2 = cross_val_score_model_r2.mean()
print(f"Cross-validation R²: {cross_val_r2:.4f}")

Cross-validation RMSE: 1.9592
Cross-validation R²: 0.5946
